# Synchronisation

Basically synchronization is needed so that there isn't a race between the coros. And for that we have the following, however it is important to have in mind that not most of the following work as intended, some aren't totally complete yet.

## Lock

A lock basically lets you put other functions in a wait state before it is released, meaning they are acquired by a function, and the other ones that wait for it, can't continue before it is released, this way all the functions can run one after another without problems. Of course they don't necesarily run in a desired order.

Here we will show some examples, first we must connect with the board.

In [1]:
%serialconnect --port /dev/ttyUSB0 --baudrate 115200

Connecting to --port=/dev/ttyUSB0 --baud=115200 could not open port /dev/ttyUSB0: [Errno 2] No such file or directory: '/dev/ttyUSB0'

Are you sure your ESP-device is plugged in?

Then first, uasyncio and Lock must be imported, as well as our function which will let us show how lock works and a killer function which defines a time in specific in which the processes will run.

In [ ]:
async def task(i, lock):
    while 1:
        await lock.acquire()
        print("Acquired lock in task", i)
        await asyncio.sleep(0.5)
        lock.release()

async def killer():
    await asyncio.sleep(10)

Then we initialize our `Lock` as follows, were as a parameter you can define a delay in ms., Remember it would be considered as a global instance.

In [ ]:
lock = Lock()

In [ ]:
And last, we add the desired tasks an run it.

In [ ]:

loop.create_task(task(1, lock))
loop.create_task(task(2, lock))
loop.create_task(task(3, lock))

loop.run_until_complete(killer()) 

## Event

The event class is kinda like `Lock` one, the difference is that `Lock` puts all coros in some sort of order in which just one of them is run after the other, however event lets us stops on or more at the same time, and then let them continue at the same time. In order to use event, we should then import `asyn`.

In [ ]:
import asyn
event = asyn.Event()

In [ ]:
async def eventwait(event):
    print('waiting for event')
    await event
    print('got event')
    event.clear()
loop = asyncio.get_event_loop()
loop.create_task(eventset(event))
await eventwait(event)

## Barrier

This one basically lets a coro wait for another function to run before continuing. In this case some parameters must be specified like the amount of coros that will use it, the callback and the parameters of the callback. Lets run an example:


In [2]:
import asyn

def callback(text):
    print(text)

barrier = asyn.Barrier(3, callback, ('Synch',))

async def report():
    for i in range(5):
        print('{} '.format(i), end='')
        await barrier

No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands

## Semaphore

Semaphore, different from `Locks`, lets us put in queue coros, and in some way just a defined amount of them can keep running, just when the object semaphore is released. Lets run an example to understand it more.

In [ ]:
import asyncio
import time

async def myWorker(semaphore):
    await semaphore.acquire()
    print("Successfully acquired the semaphore")
    await asyncio.sleep(3)
    print("Releasing Semaphore")
    semaphore.release()

async def main(loop):
    mySemaphore = asyncio.Semaphore(value=2)
    await asyncio.wait([myWorker(mySemaphore), myWorker(mySemaphore), myWorker(mySemaphore)])
    print("Main Coroutine")

loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))
print("All Workers Completed")
loop.close()

## Queue

At last we have `Queue`, which lets us to pass values between coros. If the object Queue doesn't have anything inside, what is returned is None.

In [ ]:
from uasyncio.queues import Queue
q = Queue()
async def producer(q):
    while True:
        result = await slow_process()  # somehow get some data
        await q.put(result) 
        

async def consumer(q):
    while True:
        result = await(q.get())  # Will pause if q is empty
        print('Result was {}'.format(result))
